In [519]:
import pandas as pd
import re, nltk, random, os, json
from nltk.corpus import stopwords
from rank_bm25 import BM25Okapi, BM25L, BM25Plus

In [524]:
from simpletransformers.question_answering import QuestionAnsweringModel
import json
import os
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Create the QuestionAnsweringModel
model = QuestionAnsweringModel('distilbert', 'distilbert-base-uncased-distilled-squad', use_cuda=False, args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'use_cuda': False})

INFO:filelock:Lock 7537684720 acquired on /Users/jarana/.cache/torch/transformers/e88f38f2c8bc669ef7873de68f36bf764d4f64b9833ca8401efe271aab476745.0f15800a5b4c30725c555e054e3d0262e9916635f0de9d397c30acd86c21dc73.lock


INFO:filelock:Lock 7537684720 released on /Users/jarana/.cache/torch/transformers/e88f38f2c8bc669ef7873de68f36bf764d4f64b9833ca8401efe271aab476745.0f15800a5b4c30725c555e054e3d0262e9916635f0de9d397c30acd86c21dc73.lock


In [531]:
# idx = 0
# to_predict = []
# for context in context_sentences:
#     qas = []
#     for prop in cand_properties:
#         qas.append({'question': cand_properties[prop], 'id': idx})
#         idx += 1
#     to_predict.append({'context': context, 'qas':qas})

In [521]:
stop_words = set(stopwords.words('english'))

regex = re.compile('[^a-zA-Z0-9.,]')
#First parameter is the replacement, second parameter is your input string
def preprocessingText(doc,enableStopword=False):
#     doc = regex.sub(' ', doc)
    if enableStopword:
        doc = " ".join([w for w in doc.split() if not w in stop_words])
    return doc.lower().replace("\n","")

df_wiki = pd.read_csv("data/wikihowSep.csv")
df_wiki['title'] = df_wiki['title'].str.replace("How to ", "")
for col in ['overview', 'headline', 'text', 'sectionLabel', 'title']:
    df_wiki[col] = [preprocessingText(str(i), False) for i in df_wiki[col]]
    
df_wiki['title'] = [i if not i[-1].isdigit() else i[:-1] for i in df_wiki['title']]

In [ ]:
1234

In [64]:
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

INFO:pytorch_pretrained_bert.modeling:Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/jarana/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [611]:
import glob, re, math

def camel_case_split(str): 
    words = [[str[0]]] 
    for c in str[1:]: 
        if words[-1][-1].islower() and c.isupper(): 
            words.append(list(c)) 
        else: 
            words[-1].append(c) 

    return " ".join([i.lower() for i in [''.join(word) for word in words]])    

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext.lower()

def simplifyDomainName(oldName):
    special = {"buse": "bus", "rentalcar": "rent car", "ridesharing": "taxi", "messaging": "message"}
    newName = oldName.split("_")[0].lower()
    if newName[-1] == "s":
        newName = newName[:-1]
    if newName in special:
        newName = special[newName]
    return newName

schema = pd.read_json("data/dialog/dstc8-schema-guided-dialogue/train/schema.json")
schema = pd.concat([schema, pd.read_json("data/dialog/dstc8-schema-guided-dialogue/dev/schema.json")])
schema = pd.concat([schema, pd.read_json("data/dialog/dstc8-schema-guided-dialogue/test/schema.json")])

schema['service_name'] = [simplifyDomainName(i) for i in schema.service_name.tolist()]
domains = schema.service_name.unique().tolist()

orgType = pd.read_csv("data/dialog/all-layers-types.csv")
orgProp = pd.read_csv("data/dialog/all-layers-properties.csv")

schemaCorpus = [camel_case_split(i[0]) + " #SEP# " + cleanhtml(i[1]).lower() for i in orgType[['label', 'comment']].values]
bm25s = BM25L([doc.split() for doc in schemaCorpus])

sProp2Desc = {camel_case_split(i[0]):cleanhtml(i[1]).lower() for i in np.concatenate([orgProp[['label', 'comment']].values, orgType[['label', 'comment']].values])}
sType2Prop = {}
sType2Subtype = {}
for i,j,k in orgType[['label', 'subTypeOf', 'properties']].values:
    if type(j) == str:
        sType2Subtype[camel_case_split(i)] = [camel_case_split(z) for z in j.replace("http://schema.org/", "").split(", ")]
                           
    if type(k) == str:
        sType2Prop[camel_case_split(i)] = [camel_case_split(z) for z in k.replace("http://schema.org/", "").split(", ")]
    

In [533]:
corpus = []
doc2title = {}
for name, group in df_wiki.groupby("title"):
    doc = []
    for col in ["overview", "sectionLabel", "headline", "text"]:
        doc.append(" ".join(group[col].unique().tolist()))
        
    doc = name+" #SEP# "+ " ".join(doc)
    corpus.append(doc)
    doc2title[doc] = name
bm25w = BM25L([doc.split() for doc in corpus])

In [456]:
# distant
from textblob import TextBlob
import nltk
unknown_services = ['alarm', 'weather']
matches = []
retrieved_properties = []
for service, desc in schema[['service_name', 'description']].values:
#   ignore weather and alarm
    if service in unknown_services:
        continue
        
    types = [i.split(" #SEP# ")[0] for i in bm25s.get_top_n(service.split(), schemaCorpus, n=5)]
    wiki2desc = dict(i.split(" #SEP# ") for i in bm25w.get_top_n(preprocessingText(desc).split(), corpus, n=5))

    #   get subtypes
    subtypes = []
    for t in types:
        if t in sType2Subtype:
            subtypes.extend(sType2Subtype[t])
    subtypes = list(set(subtypes))
    
    cand_props = []
    for t in types:
        if t in sType2Prop:
            cand_props.extend(sType2Prop[t])
            retrieved_properties.extend(sType2Prop[t])
    cand_props += types
    
    seen_prop = [""]
    for prop in cand_props:
        if prop in schema.service_name.unique().tolist():
            continue
        propLast = prop.split()[-1]
        #   check if nous
        tokenized = nltk.word_tokenize(propLast)
        nouns = [word  for (word, pos) in nltk.pos_tag(tokenized) if(pos[:2] == 'NN')]
        if len(nouns) == 0:
            propLast = ""
            
        for wiki in wiki2desc:
            for sentence in wiki2desc[wiki].split(". "):
                found_exact_match = False
                exact_match_start = sentence.find(prop)
                if exact_match_start > -1:
                    matches.append([sProp2Desc[prop], prop, exact_match_start, sentence])
                    found_exact_match = True
                    
                if propLast not in seen_prop and not found_exact_match:

                    exact_match_start = sentence.find(propLast)
                    if exact_match_start > -1:
                        matches.append([sProp2Desc[prop], propLast, exact_match_start, sentence])
        seen_prop.append(propLast)

In [439]:
idx = 1
trainData = []
for i in matches:
    question, text, answer_start, context = i
    neg_question = random.choice(list(sProp2Desc.values()))
    while neg_question == question:
        neg_question = random.choice(list(sProp2Desc.values()))
                        
    tmp = {"context": context, 'qas':[{'id':str(idx+1), 'is_impossible': True, 'question': neg_question, 'answers':[]},{'id':str(idx), 'is_impossible': False, 'question': question, 'answers':[{'text': i[1], 'answer_start': i[2]}]}]}
    trainData.append(tmp)
    idx += 2

os.makedirs('data', exist_ok=True)
with open('data/dialog/distantWithNegativeTrain.json', 'w') as f:
    json.dump(trainData, f)

In [441]:
# model.train_model('data/dialog/distantWithNegativeTrain.json')

In [649]:
single_domain_dialogs.head()

,dialogue_id,services,turns,service_name,description,slots,intents,domain
0,1100000.0,[Events_2],"[{'frames': [{'service': 'Events_2', 'slots': ...",NaN,NaN,NaN,NaN,Events_2
1,1100001.0,[Events_2],"[{'frames': [{'service': 'Events_2', 'slots': ...",NaN,NaN,NaN,NaN,Events_2
2,1100002.0,[Events_2],"[{'frames': [{'service': 'Events_2', 'slots': ...",NaN,NaN,NaN,NaN,Events_2
3,1100003.0,[Events_2],"[{'frames': [{'service': 'Events_2', 'slots': ...",NaN,NaN,NaN,NaN,Events_2
4,1100004.0,[Events_2],"[{'frames': [{'service': 'Events_2', 'slots': ...",NaN,NaN,NaN,NaN,Events_2


In [655]:
# generate dstc8-schema-guided-dialogue
# Train and Test set
from sklearn.model_selection import train_test_split
import random
import glob

# read dialog datasets
txtfiles = []
dialog = None
for i in ['train', 'dev', 'test']:
    for file in glob.glob("data/dialog/dstc8-schema-guided-dialogue/%s/*.json" % i):
        if "schema.json" in file:
            continue
        txtfiles.append(file)
        dialog = pd.concat([dialog, pd.read_json(file)])

# # extract single domains
unique_single_domains = list(set([simplifyDomainName(i[0]) for i in dialog[dialog.services.str.len() == 1]['services']]))
# split domains to train and test sets
trainIds, testIds = train_test_split(list(set(unique_single_domains)), test_size=0.5, random_state=2020)

train_data = []
test_data = []

prop2desc = {}
schemaT = pd.read_json("data/dialog/dstc8-schema-guided-dialogue/train/schema.json")
schemaT = pd.concat([schemaT, pd.read_json("data/dialog/dstc8-schema-guided-dialogue/dev/schema.json")])
schemaT = pd.concat([schemaT, pd.read_json("data/dialog/dstc8-schema-guided-dialogue/test/schema.json")])
for idx, s in schemaT.iterrows():
    service = s.service_name
    for i in s.slots:
        prop2desc[service+"_"+i['name']] = i['description']


single_domain_dialogs = dialog[dialog.services.str.len() == 1]
# single_domain_dialogs['domain'] = [i[0] for i in single_domain_dialogs['services']]

qid = 1
# for idx, row in dialog.iterrows():
# for idx, row in tmp.groupby('domain',as_index = False).apply(lambda x: x.sample(20)).iterrows():
for idx, row in single_domain_dialogs.iterrows():
    frames = row['turns']
    context = []
    
    if str(frames)=='nan':
        continue
        
    for f in frames:
        service = f['frames'][0]['service']
#         service = service.split("_")[0].lower()
        if len(f['frames']) == 1:
            utterance = f['utterance']
            if len(f['frames'][0]['slots']) != 0:
                terms = []
                for s in f['frames'][0]['slots']:
                    
#                   replace text with slot name
#                     _context = " ".join(context)
#                     answer_start = len(_context) + s["start"] + 1
#                     new_answer = utterance[:s["start"]] + s['slot'].replace("_", " ") + utterance[s["exclusive_end"]+1:]
#                     _context += " " + new_answer
#                     _json = {'context': _context, 'qas':[{'id': str(qid), 'is_impossible': False, 
#                                                           'question': prop2desc[service+"_"+s['slot']], 'answers':[{'text':s['slot'].replace("_"," "), 
#                                                            'answer_start': answer_start}]}]}
                
                    _context = " ".join(context)
                    answer_start = len(_context) + s["start"] + 1
#                     new_answer = utterance[:s["start"]] + s['slot'].replace("_", " ") + utterance[s["exclusive_end"]+1:]
                    _context += " " + utterance
                    question = prop2desc[service+"_"+s['slot']]
#                   random negative question
                    neg_question = random.choice(list(prop2desc.values()))
                    while neg_question == question:
                        neg_question = random.choice(list(prop2desc.values()))
                        
                    _json = {'context': _context, 'qas':[{'id': str(qid), 'is_impossible': False, 
                                                          'question': question, 'answers':[{'text':utterance[s['start']: s['exclusive_end']], 
                                                           'answer_start': answer_start}]}, {'id': str(qid+1), 'is_impossible': True, 'question': neg_question, 'answers':[] }]}
    
                    if simplifyDomainName(service) in trainIds:
                        train_data.append(_json)
                    else:
                        test_data.append(_json)
                    qid += 2
            context.append(utterance)

In [657]:
len(train_data), len(test_data)

(34461, 38913)

In [658]:
### Save as a JSON file
import json
with open('data/dialog/defaultWithNegativeTrain.json', 'w') as f:
    json.dump(train_data, f)
import json
with open('data/dialog/defaultWithNegativeTest.json', 'w') as f:
    json.dump(test_data, f)

In [432]:
# model.train_model('data/dialog/defaultWithNegativeTrain.json')